# Set up OpenAI and start with a system prompt

In [4]:
import json
import os

from openai import OpenAI

In [5]:
EXAMPLES_DIR = 'examples_validation'
os.makedirs(EXAMPLES_DIR, exist_ok=True)

In [6]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=open('../key.txt', 'r').read().strip('\n'),
)

In [7]:
system_prompt = """You are AutocompleteGPT, a useful AI autocomplete tool that provides code completions based on the user's code.
You are a precision-focused tool for code autocompletion, adept in languages like Python, JavaScript, C++, and SQL.
Precisely continue the code from the point of interruption and do not repeat or modify the original code, even if it is incorrect or the code interrupts in the middle of a line.
Your code is well documented with comments and annotations, and you should provide a clear explanation of the code's purpose in your code completion.
Your unique capability is to provide completions without altering, repeating, or commenting on the original code.
You offer only the necessary code to complete the snippet, ensuring the response is exclusively code, with no additional comments, explanations, or annotations.
This approach makes you an ideal assistant for users seeking straightforward and efficient code extensions, enhancing their work with accurate, logic-driven completions while maintaining the integrity and simplicity of the original input.
Your response begins with the next characters of the line if the last line of the user'S code is incomplete, or the next line if the last line of the user's code is complete.
Your application is a VSCode extension like GitHub Copilot, which provides seamless code completions based on the user's code at the point of interruption.
"""

In [8]:
def add_message(messages: list[str], role: str, message: str | None = None, model: str = "gpt-3.5-turbo") -> list[str]:
    # If the user is the assistant, generate a response
    if role == "assistant" and message is None:
        chat_completion = client.chat.completions.create(messages=messages, model=model)
        message = chat_completion.choices[0].message.content

    # Add the message to the messages list
    messages.append({
        "role": role,
        "content": message,
    })

    return messages

In [28]:
user_input = '''import argparse
import logging
import os

from ..utils import construSct_predictions_index_name, get_es_client

logger = logging.getLogger('pmtrendviz.list')


def list_managers() -> None:
    """
    List the implemented managers
    """
    from ..models import manager as mgr

    # Get all classes defined in the module
    classes = [getattr(mgr, name) for name in dir(mgr) if isinstance(getattr(mgr, name), type)]

    # Filter the classes to only get the managers
    managers = [cls for cls in classes if issubclass(cls, mgr.ModelManager) and not cls == mgr.ModelManager]

    # Collect the names of the managers
    managers_list = [manager.__name__ '''

# Base GPT-3.5-Turbo

In [29]:
messages = []

In [30]:
messages = add_message(messages, "system", system_prompt)
messages = add_message(messages, "user", user_input)

In [31]:
messages = add_message(messages, "assistant")

In [32]:
print(messages[-1]['content'])

managers_list = [manager.__name__ for manager in managers]

    # Print the list of managers
    for manager_name in managers_list:
        print(manager_name)

if __name__ == '__main__':
    # Parse command line arguments
    parser = argparse.ArgumentParser()
    parser.add_argument('--host', type=str, default='localhost', help='Elasticsearch host')
    parser.add_argument('--port', type=int, default=9200, help='Elasticsearch port')
    args = parser.parse_args()

    # Set up logging
    logging.basicConfig(level=logging.INFO)

    # Get Elasticsearch client
    es_client = get_es_client(args.host, args.port)

    # List the managers
    list_managers()


# Fine-Tuned GPT-3.5-Turbo

In [33]:
messages = []

In [34]:
messages = add_message(messages, "system", system_prompt)
messages = add_message(messages, "user", user_input)

In [35]:
messages = add_message(messages, "assistant", message=None, model="ft:gpt-3.5-turbo-1106:personal::8LCi9Q0d")

In [36]:
print(messages[-1]['content'])

for manager in managers]

    # Print the managers
    for manager_name in managers_list:
        print(manager_name)


# Compare

In [163]:
comment = """
- results in invalid url (e.g. 'https://eur-lex.europa.eu/legal-content/AUTO/?uri=CELEX:31983H0230&=1698155004044&=460') because it only removes the parameters but not their values


Ground Truth:
```python
url = url.replace('AUTO', 'EN/ALL')
    return url[:url.find('&')]
```
"""

title = "Truncate URL gone wrong"

In [164]:
# Remove any previous comments
messages = [message for message in messages if message['role'] != 'comment']

# Add the comment to the messages list
messages.append({
    "role": "comment",
    "content": comment,
})

In [165]:
# Save the messages to a file
if not os.path.exists(os.path.join(EXAMPLES_DIR, f"{title}.json")):
    with open(os.path.join(EXAMPLES_DIR, f"{title}.json"), "w") as f:
        json.dump(messages, f, indent=4, ensure_ascii=False)
else:
    print(f"File {title}.json already exists. Skipping.")